In [1]:
import spires
import scipy
import netCDF4
import numpy as np
import matplotlib.pyplot as plt

# Random

In [2]:
lut_file = '../tests/data/lut_sentinel2b_b2to12_3um_dust.mat' 

with netCDF4.Dataset(lut_file) as lut_nc:
    reflectances = np.squeeze(lut_nc['#refs#']["h"][:])
    grain_sizes = np.squeeze(lut_nc['#refs#']['d'][:])
    dust_concentrations = np.squeeze(lut_nc['#refs#']['e'][:])
    solar_angles = np.squeeze(lut_nc['#refs#']['f'][:])
    bands = np.squeeze(lut_nc['#refs#']['g'][:])

#dust_concentrations = dust_concentrations[2:]
#reflectances = reflectances[:, :, 2:, :]
#reflectances = np.ascontiguousarray(reflectances)

In [3]:
truth = reflectances[:,0,1,0]
truth

masked_array(data=[0.99236893, 0.987902  , 0.97464906, 0.96756319,
                   0.96042174, 0.94498655, 0.1533866 , 0.18644477,
                   0.92160772],
             mask=False,
       fill_value=1e+20)

In [4]:
solar_angle = 0
dust_concentration = 0.1
grain_size = 30

#solat_angle = 55.73733298
#dust_concentration = 491. 
#grain_size = 550.  

# New Swig interpolator

In [5]:
interpolator = spires.LutInterpolator(reflectances=reflectances, bands=bands, solar_angles=solar_angles, dust_concentrations=dust_concentrations, grain_sizes=grain_sizes)

In [6]:
##%%timeit
result = []
for band_id in range(1,10):
    ip = interpolator.interpolate(band=band_id, solar_angle=solar_angle, grain_size=grain_size, dust_concentration=dust_concentration)
    result.append(ip)
result

[0.9923689336510294,
 0.9879020029447392,
 0.9746490633363271,
 0.9675631898383531,
 0.9604217444912628,
 0.9449865463225621,
 0.15338659686190492,
 0.18644476956696648,
 0.9216077180306237]

In [7]:
np.array_equal(truth, result)

True

In [8]:
%%timeit
interpolator.interpolate_all(solar_angle=solar_angle, grain_size=grain_size, dust_concentration=dust_concentration)

272 ns ± 9.42 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [9]:
%%timeit
interpolator.interpolate_all_np_index(solar_angle=solar_angle, grain_size=grain_size, dust_concentration=dust_concentration)

7.05 µs ± 373 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Standard Scipy interpolator

In [10]:
points = [bands, solar_angles, dust_concentrations, grain_sizes]
interpolator_scipy = scipy.interpolate.RegularGridInterpolator(points=points, values=reflectances)

In [11]:
interpolator_scipy([1, 1, dust_concentration, grain_size])

array([0.99236963])

In [12]:
#%%timeit
result = []
for band_id in range(1,10):
    ip = interpolator_scipy([band_id, solar_angle, dust_concentration, grain_size])
    result.append(ip)
result = np.array(result).squeeze()
result

array([0.99236893, 0.987902  , 0.97464906, 0.96756319, 0.96042174,
       0.94498655, 0.1533866 , 0.18644477, 0.92160772])

In [13]:
np.allclose(truth, result)

True

# Scipy interpolator w/o bands

In [14]:
lut_interpolator_= {}
for b in bands:
    b = int(b)-1
    lut_interpolator_[b] = scipy.interpolate.RegularGridInterpolator(points=[solar_angles, dust_concentrations, grain_sizes], values=reflectances[b])

def lut_interpolator_f(pts):
    band = int(pts[0])-1
    return lut_interpolator_[band](pts[1:])

In [15]:
#%%timeit
result = []
for band_id in range(1,10):
    ip = lut_interpolator_f([band_id, solar_angle, dust_concentration, grain_size])
    result.append(ip)
result

[array([0.99236893]),
 array([0.987902]),
 array([0.97464906]),
 array([0.96756319]),
 array([0.96042174]),
 array([0.94498655]),
 array([0.1533866]),
 array([0.18644477]),
 array([0.92160772])]

# Vectorized Scipy

In [16]:
#%%timeit
bands = np.arange(1.0, 10)
sz = np.repeat(solar_angle, 9)
dust= np.repeat(dust_concentration, 9)
grain = np.repeat(grain_size, 9)
pts = np.vstack([bands, sz, dust, grain]).T    
result = interpolator_scipy(pts)

In [17]:
np.array_equal(truth, result)

True